In [2]:
import torch
from torch.utils import data
from os.path import join
from PIL import Image
import pandas as pd

class CSVImageDataset(data.Dataset):
    def __init__(self, data_root, csv, transform=None):
        self.data_root = data_root
        self.data = pd.read_csv(csv)
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        im_path, im_label = self.data.iloc[i]['Path'], self.data.iloc[i].Label
        # il dataset contiene alcune immagini in scala di grigi
        # convertiamo tutto in RGB per avere delle immagini consistenti
        im = Image.open(join(self.data_root, im_path)).convert('RGB')
        
        #Per fare eventualmente data augmentation o quelle come toTensor?
        if self.transform is not None:
            im = self.transform(im)
        
        return im, im_label

In [3]:
import numpy as np
from torchvision import transforms

In [4]:
resolution=640

In [5]:


preprocessing_trasnform = transforms.Compose([
    transforms.Resize((resolution,resolution)),
    transforms.ToTensor(),
])


dataset_train=CSVImageDataset('euro_dataset/train/images','train.csv',transform=preprocessing_trasnform)
dataset_valid=CSVImageDataset('euro_dataset/valid/images','valid.csv',transform=preprocessing_trasnform)
dataset_test=CSVImageDataset('euro_dataset/test/images','test.csv',transform=preprocessing_trasnform)

In [6]:
# im,lab=dataset_train[55]
im,lab=dataset_train[150]
print('#Coins:',lab)
print(im.shape)
im.mean()
im[0].mean()


#Coins: 1
torch.Size([3, 640, 640])


tensor(0.4322)

In [80]:
from torchvision.models import squeezenet1_0
from torchvision.models import SqueezeNet1_0_Weights
from torch import nn

model= squeezenet1_0(weights=SqueezeNet1_0_Weights.DEFAULT)

num_class=1
model.classifier[1]=nn.Conv2d(512,num_class,kernel_size=(1,1),stride=(1,1))
model.num_classes=num_class
model

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

# MEDIA E VARIANZA DA CALCOLARE

In [73]:

red_mean=0
green_mean=0
blue_mean=0
for image in dataset_train:
    red_mean+=image[0][0].sum()# somma di tutti i pixel
    green_mean+=image[0][1].sum()# somma di tutti i pixel
    blue_mean+=image[0][2].sum()# somma di tutti i pixel

#dividiamo per il numero di immagini molt numero di pixel
red_mean=red_mean/(len(dataset_train)*(resolution)**2)
green_mean=green_mean/(len(dataset_train)*(resolution**2))
blue_mean=blue_mean/(len(dataset_train)*(resolution**2))

#deviazione standard
red_std=0
green_std=0
blue_std=0
for image in dataset_train:
    red_std+=((image[0][0]-red_mean)**2).sum()
    green_std+=((image[0][1]-green_mean)**2).sum()
    blue_std+=((image[0][2]-blue_mean)**2).sum()

#sqrt della varianza
red_std=np.sqrt(red_std/(len(dataset_test)*(resolution)**2))
green_std=np.sqrt(green_std/(len(dataset_test)*(resolution**2)))
blue_std=np.sqrt(blue_std/(len(dataset_test)*(resolution**2)))

In [74]:
print(red_mean,red_std)
print(green_mean,green_std)
print(blue_mean,blue_std)

tensor(0.6418) tensor(0.6095)
tensor(0.6030) tensor(0.6272)
tensor(0.5569) tensor(0.6531)


In [75]:
from torchvision import transforms
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([red_mean, green_mean, blue_mean], [red_std, green_std, blue_std])
])

from torchvision import transforms
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),##?????
    transforms.ToTensor(),
    transforms.Normalize([red_mean, green_mean, blue_mean], [red_std, green_std, blue_std])
])

In [76]:
from torch.utils.data import DataLoader
dataset_train=CSVImageDataset('euro_dataset/train/images','train.csv',transform=train_transform)
dataset_valid=CSVImageDataset('euro_dataset/valid/images','valid.csv',transform=test_transform)
dataset_test=CSVImageDataset('euro_dataset/test/images','test.csv',transform=test_transform)

caltech101_train_loader=(DataLoader(dataset_train,batch_size=32,num_workers=0,shuffle=True))
caltech101_valid_loader=(DataLoader(dataset_valid,batch_size=32,num_workers=0))
caltech101_test_loader=(DataLoader(dataset_test,batch_size=32,num_workers=0))

In [77]:
from torch.optim import SGD
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score
from os.path import join
import os


class AverageValueMeter():
    def init(self):
        self.reset()

    def reset(self):
        self.sum = 0
        self.num = 0

    def add(self, value, num):
        self.sum += value*num
        self.num += num

    def value(self):
        try:
            return self.sum/self.num
        except:
            return None

In [81]:
def trainval(model, train_loader, test_loader, exp_name='experiment', lr=0.01, epochs=10, momentum=0.99, logdir='logs'):
    criterion = nn.MSELoss()
    optimizer = SGD(model.parameters(), lr, momentum=momentum)
    # meters
    loss_meter = AverageValueMeter()
    acc_meter = AverageValueMeter()
    # writer
    writer = SummaryWriter(join(logdir, exp_name))
    # device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    # definiamo un dizionario contenente i loader di training e test
    loader = {
        'train': train_loader,
        'test': test_loader
    }
    # inizializziamo il global step
    global_step = 0
    os.makedirs("weights", exist_ok=True)

    for e in range(epochs):
        print(f"Epoch {e+1} of {epochs}")
        # iteriamo tra due modalità: train e test
        for mode in ['train']:  #, 'test'
            loss_meter.reset()
            acc_meter.reset()
            model.train() if mode == 'train' else model.eval()
            with torch.set_grad_enabled(mode=='train'): # abilitiamo i gradienti solo in training
                for i, batch in enumerate(loader[mode]):
                    x = batch[0].to(device) # portiamoli sul device corretto
                    y = batch[1].to(device)

                    y=y.float()

                    output = model(x)
                    print(y.dtype)
                    print(output.dtype)
                    # aggiorniamo il global step
                    # conterrà il numero di campioni visti durante il training
                    n = x.shape[0] # numero di elementi nel batch
                    global_step += n
                    print(output.view(-1).shape,y.shape)
                    l = criterion(output.view(-1), y)
                    print(l)
                    if mode=='train':
                        l.backward()
                        optimizer.step()
                        optimizer.zero_grad()
                    
                    acc = accuracy_score(y.to('cpu'), output.to('cpu').max(1)[1])
                    loss_meter.add(l.item(), n)
                    acc_meter.add(acc, n)
                    
                    # Loggiamo i risultati iterazione per iterazione solo durante il training
                    if mode=='train':
                        writer.add_scalar('loss/train', loss_meter.value(), global_step=global_step)
                        writer.add_scalar('accuracy/train', acc_meter.value(), global_step=global_step)
            
            # una volta finita l'epoca (sia nel caso di training che test, loggiamo le stime finali)
            writer.add_scalar('loss/' + mode, loss_meter.value(), global_step=global_step)
            writer.add_scalar('accuracy/' + mode, acc_meter.value(), global_step=global_step)
            
            # conserviamo i pesi del modello alla fine di un ciclo di training e test
            torch.save(model.state_dict(), '%s-%d.pth'%(exp_name, e+1))
    return model

In [82]:
squeezenet_caltech101_finetuned=trainval(model,caltech101_train_loader,caltech101_valid_loader,exp_name='squeezenet_caltech101_finetuning',lr=0.001,epochs=5)

Epoch 1 of 5
torch.float32
torch.float32
torch.Size([32]) torch.Size([32])
tensor(20.8369, grad_fn=<MseLossBackward0>)


KeyboardInterrupt: 